# This is an exploratory file for Budnitz RA work

In [ ]:
# import libraries
!pip install pandas
import pandas as pd
pd.set_option('display.max_columns', 50)

import os
from urllib.request import urlopen
import json
!pip install plotly
import plotly.express as px
!pip install matplotlib
import matplotlib.pyplot as plt
!pip install seaborn
import seaborn as sns
!pip install numpy
import numpy as np
!pip install geopandas
import geopandas as gpd
from shapely.geometry import shape
from collections import defaultdict

!pip install --upgrade nbformat
import nbformat

In [2]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', 50)

import os
from urllib.request import urlopen
import json
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from collections import defaultdict
import nbformat

In [3]:
# survey data import

# Get the directory of the current notebook
parent_dir = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("__file__")), os.pardir))
filepath_local_authority = os.path.join(parent_dir, 'raw_data', 'LAoutputFleet.csv')

# read LSOA and Local Authority data
df_loc_auth = pd.read_csv(filepath_local_authority)
df_msoa = pd.read_csv(os.path.join(parent_dir, 'raw_data', 'MSOAOutput.csv'))
df_msoa = df_msoa.drop(columns=['Unnamed: 0'])

In [14]:
# geojson data import

# tolerance for simplifying geojson
tolerance = 0.005

# geojson with LAD22CD and LAD22NM
geojson_lad22nm_path = os.path.join(parent_dir, 'raw_data', 'Local_Authority_Districts_December_2022_UK_BFC_V2_-7857726022236933518.geojson')
simplified_geojson_lad22nm_path = os.path.join(parent_dir, 'dashboard', 'cleaned_data', 'simplified_geojson_lad22nm.geojson')
geojson_lad22nm = gpd.read_file(geojson_lad22nm_path)
geojson_lad22nm['geometry'] = geojson_lad22nm['geometry'].simplify(tolerance, preserve_topology=True)
geojson_lad22nm.to_file(simplified_geojson_lad22nm_path, driver='GeoJSON')
with open(simplified_geojson_lad22nm_path) as response:
    geojson_lad22nm = json.load(response)

# geojson that matches df_loc_auth
geojson_loc_auth_path = os.path.join(parent_dir, 'raw_data', 'Counties_and_Unitary_Authorities_December_2024_Boundaries_UK_BFC_3348038940373313033.geojson')
simplified_geojson_loc_auth_path = os.path.join(parent_dir, 'dashboard', 'cleaned_data', 'simplified_geojson_loc_auth.geojson')
geojson_loc_auth = gpd.read_file(geojson_loc_auth_path)
geojson_loc_auth['geometry'] = geojson_loc_auth['geometry'].simplify(tolerance, preserve_topology=True)
geojson_loc_auth.to_file(simplified_geojson_loc_auth_path, driver='GeoJSON')
with open(simplified_geojson_loc_auth_path) as response:
    geojson_loc_auth = json.load(response)

# geojson with MSOA21CD
geojson_msoa_path = os.path.join(parent_dir, 'raw_data', 'Middle_layer_Super_Output_Areas_December_2021_Boundaries_EW_BFC_V7_303696399389513507.geojson')
simplified_geojson_msoa = os.path.join(parent_dir, 'dashboard', 'cleaned_data', 'simplified_geojson_msoa.geojson')
geojson_msoa = gpd.read_file(geojson_msoa_path)
geojson_msoa['geometry'] = geojson_msoa['geometry'].simplify(tolerance, preserve_topology=True)
geojson_msoa.to_file(simplified_geojson_msoa, driver='GeoJSON')
with open(simplified_geojson_msoa) as response:
    geojson_msoa = json.load(response)

## Grouping df_msoa to MSOA21CD and LAD22CD each

In [6]:
# subset df_msoa to msoa columns only
df_msoa_keep_cols = ['LAD22CD', 'LAD22NM', 'MSOA21CD', 'TotHholdMSOA', 'VehsRegMSOA', 'MSOACarOwnRates', 'EVRegMSOA',
                    'EVRateMSOA', 'CarRedincCC', 'CCCarOwnMSOA', 'CCredCarOwnMSOA',
                    'MSOAEVincCC', 'MSOAEVRateincCC', 'MSOAChangeEVuptake']
df_msoa_ = df_msoa[df_msoa_keep_cols].copy()
df_msoa_ = df_msoa_.groupby('MSOA21CD').first().reset_index()

df_lad_keep_cols = ['LAD22CD', 'LAD22NM', 'TotHholdLAD', 'VehsRegLAD', 'LADCarOwnRates', 'EVRegLAD', 'EVRateLAD',
                    'LADCarRedincCC', 'CCCarOwnLAD', 'CCredCarOwnLAD', 'LADEVincCC',
                    'LADEVRateincCC', 'LADChangeEVuptake']
df_lad_ = df_msoa[df_lad_keep_cols].copy()
df_lad_ = df_lad_.groupby('LAD22CD').first().reset_index()

df_msoa = df_msoa_.copy()
df_lad = df_lad_.copy()

In [ ]:
# align column names

colnames = {'TotHholdLAD': 'TotHhold', 'VehsRegLAD': 'VehsReg', 'LADCarOwnRates': 'CarOwnRates', 'EVRegLAD': 'EVReg',
            'EVRateLAD': 'EVRate', 'LADCarRedincCC': 'CarRedincCC', 'CCCarOwnLAD': 'CCCarOwn', 'CCredCarOwnLAD': 'CCredCarOwn',
            'LADEVincCC': 'EVincCC', 'LADEVRateincCC': 'EVRateincCC', 'LADChangeEVuptake': 'ChangeEVuptake'}

df_lad = df_lad.rename(columns=colnames)

In [ ]:
# align column names

colnames = {'TotHholdMSOA': 'TotHhold', 'VehsRegMSOA': 'VehsReg', 'MSOACarOwnRates': 'CarOwnRates', 'EVRegMSOA': 'EVReg', 
            'EVRateMSOA': 'EVRate', 'CarRedincCC': 'CarRedincCC', 'CCCarOwnMSOA': 'CCCarOwn', 'CCredCarOwnMSOA': 'CCredCarOwn',
            'MSOAEVincCC': 'EVincCC', 'MSOAEVRateincCC': 'EVRateincCC', 'MSOAChangeEVuptake': 'ChangeEVuptake',
            'meanAddJobs_15byCC': 'AddJobs_15byCC', 'meanAddJobs_30byCC': 'AddJobs_30byCC', 'meanTimeCCsecSch': 'TimeCCsecSch'}

df_msoa = df_msoa.rename(columns=colnames)

In [ ]:
cd_dic = {'Somerset': 'E06000066',
          'North Yorkshire': 'E06000065',
          'Cumbria': 'E06000063'}

# replace codes in data
for nm, cd in cd_dic.items():
    df_loc_auth.loc[df_loc_auth['Local.Authority']==nm, 'Local.Authority.Code'] = cd

# duplicate data for Cumbria to replace missing row for 'Westmorland and Furness': 'E06000064'
# df_loc_auth = df_loc_auth.append(df_loc_auth[df_loc_auth['Local.Authority.Code']=='E06000063'])
waf_append = df_loc_auth[df_loc_auth['Local.Authority.Code']=='E06000063']
waf_append['Local.Authority.Code'] = 'E06000064'
waf_append['Local.Authority'] = 'Westmorland and Furness'
df_loc_auth = pd.concat([df_loc_auth, waf_append], ignore_index=True)

In [15]:
gjson_nm_ls = [geojson_lad22nm['features'][i]['properties']['LAD22NM'] for i in range(len(geojson_lad22nm['features']))]
df_nm_ls = df_lad['LAD22NM'].unique()
set(df_nm_ls) - set(gjson_nm_ls)

{'Bristol', 'Herefordshire', 'Kingston upon Hull'}

In [16]:
lad22nm_dic = {'Herefordshire': 'Herefordshire, County of',
               'Bristol': 'Bristol, City of',
               'Kingston upon Hull': 'Kingston upon Hull, City of'}

# replace names in data
for nm, nm_new in lad22nm_dic.items():
    df_lad.loc[df_lad['LAD22NM']==nm, 'LAD22NM'] = nm_new

In [17]:
df_msoa['MSOACD_LADNM'] = df_msoa['LAD22NM'] + ' (' + df_msoa['MSOA21CD'] + ')'
df_msoa = df_msoa.sort_values(by='MSOACD_LADNM')

In [ ]:
# weight reduction in car ownership and increase in EV uptake by MSOA onto LAD

def weighted_mean(group, value_column, weight_column):
    return (group[value_column] * group[weight_column]).sum() / group[weight_column].sum()

# Group by 'LAD22CD' and calculate the weighted mean of 'ChangeEVuptake' and 'CCredCarOwn' by 'TotHhold'
df_msoa['TotHhold'] = df_msoa['TotHhold'].fillna(0)
ChangeEVuptake_weight = df_msoa.groupby('LAD22CD').apply(lambda x: weighted_mean(x, 'ChangeEVuptake', 'TotHhold'))
ChangeEVuptake_weight = ChangeEVuptake_weight.reset_index()
ChangeEVuptake_weight.columns = ['LAD22CD', 'ChangeEVuptake_weight']
df_lad = df_lad.merge(ChangeEVuptake_weight, on='LAD22CD', how='left')

CCredCarOwn_weight = df_msoa.groupby('LAD22CD').apply(lambda x: weighted_mean(x, 'CCredCarOwn', 'TotHhold'))
CCredCarOwn_weight = CCredCarOwn_weight.reset_index()
CCredCarOwn_weight.columns = ['LAD22CD', 'CCredCarOwn_weight']
df_lad = df_lad.merge(CCredCarOwn_weight, on='LAD22CD', how='left')

# replace weighted columns
df_lad['ChangeEVuptake'] = df_lad['ChangeEVuptake_weight']
df_lad['CCredCarOwn'] = df_lad['CCredCarOwn_weight']
df_lad = df_lad.drop(columns=['ChangeEVuptake_weight', 'CCredCarOwn_weight'])

df_lad = df_lad.sort_values(by='LAD22NM')

df_lad

In [20]:
# save dataframes

# save df_county
df_loc_auth.to_csv(os.path.join(parent_dir, 'dashboard', 'cleaned_data', 'df_county.csv'), index=False)

# save df_lad
df_lad.to_csv(os.path.join(parent_dir, 'dashboard', 'cleaned_data', 'df_lad.csv'), index=False)

# # save df_msoa
df_msoa.to_csv(os.path.join(parent_dir, 'dashboard', 'cleaned_data', 'df_msoa.csv'), index=False)